## Question 1
- Install uv
- What's the version of uv you installed?
- Use --version to find out


In [2]:
!uv --version

uv 0.7.12 (Homebrew 2025-06-06)


## Question 2

In [5]:
# Use uv to install Scikit-Learn version 1.6.1
# What's the first hash for Scikit-Learn you get in the lock file?
# Include the entire string starting with sha256:, don't include quotes
!uv add scikit-learn==1.6.1

Resolved 125 packages in 1ms
Audited 121 packages in 0.02ms


In [9]:
!grep -i "scikit_learn-1.6.1.tar.gz" ../uv.lock

sdist = { url = "https://files.pythonhosted.org/packages/9e/a5/4ae3b3a0755f7b35a280ac90b28817d1f380318973cff14075ab41ef50d9/scikit_learn-1.6.1.tar.gz", hash = "sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e", size = 7068312, upload-time = "2025-01-10T08:07:55.348Z" }


In [10]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-27 21:39:45--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-27 21:39:46--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin’

pipeline_v1.bin

In [12]:
# verify the model checksum to be 7d17d2e4dfbaf1e408e1a62e6e880d49
!md5 pipeline_v1.bin

MD5 (pipeline_v1.bin) = 7d17d2e4dfbaf1e408e1a62e6e880d49


## Question 3

In [11]:
# Let's use the model!
# Write a script for loading the pipeline with pickle
import pickle
with open('pipeline_v1.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

# Score this record:
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}
X = dv.transform([record])
y_pred = model.predict_proba(X)[:, 1]
print(y_pred)

[0.53360727]


## Question 4

In [13]:
# Now let's serve this model as a web service
# Install FastAPI
!uv add fastapi uvicorn

Resolved 133 packages in 411ms                                       
Prepared 8 packages in 216ms                                             
Installed 8 packages in 5ms                                 
 + annotated-doc==0.0.3
 + annotated-types==0.7.0
 + fastapi==0.120.0
 + pydantic==2.12.3
 + pydantic-core==2.41.4
 + starlette==0.48.0
 + typing-inspection==0.4.2
 + uvicorn==0.38.0


In [ ]:
# Write FastAPI code for serving the model
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import uvicorn

with open('pipeline_v1.bin', 'rb') as f_in:
    pipeline = pickle.load(f_in)

app = FastAPI()
class LeadData(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

class ConvertResponse(BaseModel):
    conversion_probability: float
    is_converted: bool

def predict_single(lead_data):
    result = pipeline.predict_proba(lead_data)[0, 1]
    return float(result)


@app.post("/predict")
def predict(lead_data: LeadData) -> ConvertResponse:
    conversion_probability = predict_single(lead_data.model_dump())
    is_converted = conversion_probability >= 0.5
    return ConvertResponse(
        conversion_probability=conversion_probability,
        is_converted=is_converted
    )
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8080)

In [15]:
# Now score this client using requests:
import requests

url = "http://localhost:8080/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

requests.post(url, json=client).json()

{'conversion_probability': 0.5340417283801275, 'is_converted': True}

## Question 5

In [16]:
# Download the base image agrigorev/zoomcamp-model:2025. You can easily make it by using docker pull command.
!docker pull agrigorev/zoomcamp-model:2025

2025: Pulling from agrigorev/zoomcamp-model

9c0578b5: Pulling fs layer 
2667830b: Pulling fs layer 
2b9cc6ee: Pulling fs layer 
f5177ae2: Pulling fs layer 
357cf7a1: Pulling fs layer 
2667830b: Download complete MB/28.23MBADigest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Downloaded newer image for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025


In [18]:
# So what's the size of this base image?
!docker images | grep -i "agrigorev/zoomcamp-model"

agrigorev/zoomcamp-model   2025      14d79fde0bbf   6 days ago    181MB


**Note** : The size is 181MB. Hence the nearest option of 245 MB chosen.

## Question 6

Dockerfile

```
FROM agrigorev/zoomcamp-model:2025

COPY --from=ghcr.io/astral-sh/uv:latest /uv /uvx /bin/
WORKDIR /code

ENV PATH="/code/.venv/bin:$PATH"

COPY "pyproject.toml" "uv.lock" ".python-version" ./
RUN uv sync --locked

COPY "week5/predict-homework.py" ./predict.py

EXPOSE 8080

ENTRYPOINT ["uvicorn", "predict:app", "--host", "0.0.0.0", "--port", "8080"]

```

Build the dockerfile (on mac amd64)

```
docker build --platform linux/amd64 -t lead-prediction .
```

Run the container

```
docker run -it --rm -p 8080:8080 lead-prediction
```

In [19]:
# After running it, score this client once again:
import requests

url = "http://localhost:8080/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

requests.post(url, json=client).json()

{'conversion_probability': 0.9933071490756734, 'is_converted': True}